In [1]:
import pandas as pd
import os
import torch
from torch import nn, optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import trange

train_size = 12
predict_size = 4

Import Data file

In [2]:
# for local environment
# df = pd.read_pickle('working_dataset.pkl')

# for cloud environment
df = pd.read_csv('https://drive.google.com/uc?export=download&id='
 + '12fztRILR7EaGV-ipq_89AEAZAFUlyjUx')

data = df.iloc[:, 1:].values.astype(float)

In [3]:
# define the prediction be the % change of the prices
price_pct = [0]
for i in range(1, data.shape[0]):
    price_pct.append((data[i, 1] - data[i-1, 1])/data[i-1, 1])
price_pct = np.array(price_pct, dtype=float)
prices = data[:, 1].copy()
data[:, 1] = price_pct

# normalize the featres
normalized_features = MinMaxScaler().fit_transform(data[:, 2:])
X = np.hstack((normalized_features, np.array(price_pct).reshape(-1, 1)))
X = np.array([X[i-train_size:i, :] for i in range(train_size, len(data)-predict_size)], dtype=np.float64)
y = np.array([price_pct[i:i+predict_size] for i in range(train_size, len(data)-predict_size)], dtype=np.float64)

X.shape, y.shape

((1526, 12, 14), (1526, 4))

Build up the learning model

In [4]:
class Model(nn.Module):
    def __init__(self,hidden_size=32,num_layers=2,dropout=0.2):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(
            input_size=14,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.fc = nn.Linear(hidden_size, 4)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).requires_grad_()
        
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.fc(hn[0])
        return out

Devide The dataset into training, validating, and Testing.

In [5]:
train_X, train_y = X[:int(0.6 * len(X))], y[:int(0.6 * len(y))]
valid_X, valid_y = X[int(0.6 * len(X)):int(0.8 * len(X))], y[int(0.6 * len(y)):int(0.8 * len(y))]
test_X, test_y = X[int(0.8 * len(X)):], y[int(0.8 * len(y)):]

Applying Adam optimizer (minibatch) onto learning model

In [6]:
model = Model()

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()

min_loss = float('inf')
count = 0

for epoch in range(100):
    train_loss = 0.0

    for i in range(0, len(train_X), 16):
        output = model(torch.from_numpy(train_X[i:i+16]).float())
        loss = loss_function(output, torch.from_numpy(train_y[i:i+16]).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= (len(train_X)//16)

    # validation
    with torch.no_grad():
        valid_loss = 0.0
        for i in range(0, len(valid_X), 16):
            output = model(torch.from_numpy(valid_X[i:i+16]).float())
            loss = loss_function(output, torch.from_numpy(valid_y[i:i+16]).float())
            valid_loss += loss.item()
    valid_loss /= (len(valid_X)//16)

    print({ 'epoch': epoch, 'train_loss': '{:.2e}'.format(train_loss), 
    'valid_loss': '{:.2e}'.format(valid_loss),})

    # early stopping
    if valid_loss > min_loss:
        count += 1
        if count > 9:
            break
    else:
        min_loss = valid_loss
        count = 0
        
print(f'best score = {min_loss:.2e}')

{'epoch': 0, 'train_loss': '3.04e-03', 'valid_loss': '5.78e-04'}
{'epoch': 1, 'train_loss': '5.88e-04', 'valid_loss': '5.10e-04'}
{'epoch': 2, 'train_loss': '5.31e-04', 'valid_loss': '4.77e-04'}
{'epoch': 3, 'train_loss': '4.88e-04', 'valid_loss': '4.56e-04'}
{'epoch': 4, 'train_loss': '4.58e-04', 'valid_loss': '4.42e-04'}
{'epoch': 5, 'train_loss': '4.37e-04', 'valid_loss': '4.31e-04'}
{'epoch': 6, 'train_loss': '4.23e-04', 'valid_loss': '4.21e-04'}
{'epoch': 7, 'train_loss': '4.13e-04', 'valid_loss': '4.12e-04'}
{'epoch': 8, 'train_loss': '4.07e-04', 'valid_loss': '4.01e-04'}
{'epoch': 9, 'train_loss': '4.03e-04', 'valid_loss': '3.90e-04'}
{'epoch': 10, 'train_loss': '4.01e-04', 'valid_loss': '3.79e-04'}
{'epoch': 11, 'train_loss': '3.98e-04', 'valid_loss': '3.67e-04'}
{'epoch': 12, 'train_loss': '3.95e-04', 'valid_loss': '3.55e-04'}
{'epoch': 13, 'train_loss': '3.92e-04', 'valid_loss': '3.44e-04'}
{'epoch': 14, 'train_loss': '3.89e-04', 'valid_loss': '3.35e-04'}
{'epoch': 15, 'train

Apply fullbatch with the adam optimizer

In [7]:
model_full_batch = Model()

optimizer = optim.Adam(model_full_batch.parameters(), lr=0.001)
loss_function = nn.MSELoss()

min_loss = float('inf')
count = 0

for epoch in range(100):
    train_loss = 0.0
    output = model_full_batch(torch.from_numpy(train_X).float())
    loss = loss_function(output, torch.from_numpy(train_y).float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss = loss.item()

    # validation
    with torch.no_grad():
        valid_loss = 0.0
        output = model_full_batch(torch.from_numpy(valid_X).float())
        loss = loss_function(output, torch.from_numpy(valid_y).float())
        valid_loss = loss.item()

    print({ 'epoch': epoch, 'train_loss': '{:.2e}'.format(train_loss), 
    'valid_loss': '{:.2e}'.format(valid_loss),})

    # early stopping
    if valid_loss > min_loss:
        count += 1
        if count > 9:
            break
    else:
        min_loss = valid_loss
        count = 0

print(f'best score = {min_loss:.2e}')

{'epoch': 0, 'train_loss': '1.43e-02', 'valid_loss': '1.30e-02'}
{'epoch': 1, 'train_loss': '1.20e-02', 'valid_loss': '1.10e-02'}
{'epoch': 2, 'train_loss': '9.93e-03', 'valid_loss': '9.21e-03'}
{'epoch': 3, 'train_loss': '8.16e-03', 'valid_loss': '7.66e-03'}
{'epoch': 4, 'train_loss': '6.63e-03', 'valid_loss': '6.30e-03'}
{'epoch': 5, 'train_loss': '5.33e-03', 'valid_loss': '5.13e-03'}
{'epoch': 6, 'train_loss': '4.23e-03', 'valid_loss': '4.13e-03'}
{'epoch': 7, 'train_loss': '3.31e-03', 'valid_loss': '3.29e-03'}
{'epoch': 8, 'train_loss': '2.56e-03', 'valid_loss': '2.60e-03'}
{'epoch': 9, 'train_loss': '1.96e-03', 'valid_loss': '2.04e-03'}
{'epoch': 10, 'train_loss': '1.50e-03', 'valid_loss': '1.63e-03'}
{'epoch': 11, 'train_loss': '1.17e-03', 'valid_loss': '1.35e-03'}
{'epoch': 12, 'train_loss': '9.51e-04', 'valid_loss': '1.18e-03'}
{'epoch': 13, 'train_loss': '8.40e-04', 'valid_loss': '1.12e-03'}
{'epoch': 14, 'train_loss': '8.20e-04', 'valid_loss': '1.15e-03'}
{'epoch': 15, 'train

Applying Adam optimizer with online learning

In [8]:
model_online = Model()

optimizer = optim.Adam(model_online.parameters(), lr=0.001)
loss_function = nn.MSELoss()

min_loss = float('inf')
count = 0

for epoch in range(100):
    train_loss = 0.0

    for i in range(0, len(train_X)):
        output = model_online(torch.from_numpy(train_X[i].reshape(1, 12, 14)).float())
        loss = loss_function(output, torch.from_numpy(train_y[i].reshape(1, 4)).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= (len(train_X))

    # validation
    with torch.no_grad():
        valid_loss = 0.0
        for i in range(0, len(valid_X), 1):
            output = model_online(torch.from_numpy(valid_X[i].reshape(1, 12, 14)).float())
            loss = loss_function(output, torch.from_numpy(valid_y[i].reshape(1, 4)).float())
            valid_loss += loss.item()
    valid_loss /= (len(valid_X))

    print({ 'epoch': epoch, 'train_loss': '{:.2e}'.format(train_loss), 
    'valid_loss': '{:.2e}'.format(valid_loss),})

    # early stopping
    if valid_loss > min_loss:
        count += 1
        if count > 9:
            break
    else:
        min_loss = valid_loss
        count = 0
        
print(f'best score = {min_loss:.2e}')

{'epoch': 0, 'train_loss': '5.85e-04', 'valid_loss': '3.55e-04'}
{'epoch': 1, 'train_loss': '4.68e-04', 'valid_loss': '3.29e-04'}
{'epoch': 2, 'train_loss': '4.51e-04', 'valid_loss': '3.21e-04'}
{'epoch': 3, 'train_loss': '4.42e-04', 'valid_loss': '3.13e-04'}
{'epoch': 4, 'train_loss': '4.34e-04', 'valid_loss': '3.08e-04'}
{'epoch': 5, 'train_loss': '4.28e-04', 'valid_loss': '3.09e-04'}
{'epoch': 6, 'train_loss': '4.24e-04', 'valid_loss': '3.08e-04'}
{'epoch': 7, 'train_loss': '4.21e-04', 'valid_loss': '3.07e-04'}
{'epoch': 8, 'train_loss': '4.18e-04', 'valid_loss': '3.04e-04'}
{'epoch': 9, 'train_loss': '4.15e-04', 'valid_loss': '3.01e-04'}
{'epoch': 10, 'train_loss': '4.12e-04', 'valid_loss': '2.98e-04'}
{'epoch': 11, 'train_loss': '4.09e-04', 'valid_loss': '2.95e-04'}
{'epoch': 12, 'train_loss': '4.05e-04', 'valid_loss': '2.93e-04'}
{'epoch': 13, 'train_loss': '4.01e-04', 'valid_loss': '2.90e-04'}
{'epoch': 14, 'train_loss': '3.97e-04', 'valid_loss': '2.88e-04'}
{'epoch': 15, 'train

Trying different optimizer: SGD.

In [9]:
model = Model()
#optimizer = optim.SGD(model.parameters(), lr=0.1)
#optimizer = optim.SGD(model.parameters(), lr=0.01)
#optimizer = optim.SGD(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr=0.1,momentum = 0.2,weight_decay= 0.9)
#optimizer = optim.SGD(model.parameters(), lr=0.01,momentum = 0.5,weight_decay= 0.9)
#optimizer = optim.SGD(model.parameters(), lr=0.001,momentum = 0.8,weight_decay= 0.9)
#optimizer = optim.SGD(model.parameters(), lr=0.001,weight_decay= 0.8)
#optimizer = optim.SGD(model.parameters(), lr=0.001,weight_decay= 0.85)
optimizer = optim.SGD(model.parameters(), lr=0.001,weight_decay= 0.9)
loss_function = nn.MSELoss()

min_loss = float('inf')
count = 0

for epoch in range(100):
    train_loss = 0.0

    for i in range(0, len(train_X), 16):
        output = model(torch.from_numpy(train_X[i:i+16]).float())
        loss = loss_function(output, torch.from_numpy(train_y[i:i+16]).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= (len(train_X)//16)

    # validation
    with torch.no_grad():
        valid_loss = 0.0
        for i in range(0, len(test_X), 16):
            output = model(torch.from_numpy(valid_X[i:i+16]).float())
            loss = loss_function(output, torch.from_numpy(valid_y[i:i+16]).float())
            valid_loss += loss.item()
    valid_loss /= (len(valid_X)//16)

    print({ 'epoch': epoch, 'train_loss': '{:.2e}'.format(train_loss), 
    'valid_loss': '{:.2e}'.format(valid_loss),})

    # early stopping
    if valid_loss > min_loss:
        count += 1
        if count > 9:
            break
    else:
        min_loss = valid_loss
        count = 0

{'epoch': 0, 'train_loss': '6.71e-03', 'valid_loss': '5.85e-03'}
{'epoch': 1, 'train_loss': '5.68e-03', 'valid_loss': '4.99e-03'}
{'epoch': 2, 'train_loss': '4.85e-03', 'valid_loss': '4.30e-03'}
{'epoch': 3, 'train_loss': '4.17e-03', 'valid_loss': '3.72e-03'}
{'epoch': 4, 'train_loss': '3.60e-03', 'valid_loss': '3.24e-03'}
{'epoch': 5, 'train_loss': '3.13e-03', 'valid_loss': '2.84e-03'}
{'epoch': 6, 'train_loss': '2.74e-03', 'valid_loss': '2.50e-03'}
{'epoch': 7, 'train_loss': '2.41e-03', 'valid_loss': '2.21e-03'}
{'epoch': 8, 'train_loss': '2.13e-03', 'valid_loss': '1.97e-03'}
{'epoch': 9, 'train_loss': '1.90e-03', 'valid_loss': '1.76e-03'}
{'epoch': 10, 'train_loss': '1.70e-03', 'valid_loss': '1.57e-03'}
{'epoch': 11, 'train_loss': '1.52e-03', 'valid_loss': '1.42e-03'}
{'epoch': 12, 'train_loss': '1.37e-03', 'valid_loss': '1.28e-03'}
{'epoch': 13, 'train_loss': '1.25e-03', 'valid_loss': '1.16e-03'}
{'epoch': 14, 'train_loss': '1.14e-03', 'valid_loss': '1.06e-03'}
{'epoch': 15, 'train

For SGD optimizer, we also tried the learning rate = 0.1, 0.01, 0.001, and 0.0001. Speaking of the result, lr = 0.001 is the best match. The others are either non-convergent or having bigger loss. 

For SGD optimizer, we also tried weight decay = 0.8, 0.85, and 0.9. Speaking of the result, wd = 0.9 is the best match. The rest of them all generate bigger losses.

For SGD optimizer, we also tried momentum = 0, 0.2, 0.5, and 0.8. Speaking of the result, m = 0 is the best one having minimum loss. 

The best loss gained from SGD is 3.31e-04

Testing different optimizer: Adam (With modification of parameters comparing to original used one)

In [18]:
model = Model()
#optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay= 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_function = nn.MSELoss()

min_loss = float('inf')
count = 0

for epoch in range(100):
    train_loss = 0.0

    for i in range(0, len(train_X), 16):
        output = model(torch.from_numpy(train_X[i:i+16]).float())
        loss = loss_function(output, torch.from_numpy(train_y[i:i+16]).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= (len(train_X)//16)

    # validation
    with torch.no_grad():
        valid_loss = 0.0
        for i in range(0, len(test_X), 16):
            output = model(torch.from_numpy(valid_X[i:i+16]).float())
            loss = loss_function(output, torch.from_numpy(valid_y[i:i+16]).float())
            valid_loss += loss.item()
    valid_loss /= (len(valid_X)//16)

    print({ 'epoch': epoch, 'train_loss': '{:.2e}'.format(train_loss), 
    'valid_loss': '{:.2e}'.format(valid_loss),})

    # early stopping
    if valid_loss > min_loss:
        count += 1
        if count > 14:
            break
    else:
        min_loss = valid_loss
        count = 0
print(f'best score = {min_loss:.2e}')

{'epoch': 0, 'train_loss': '1.15e-02', 'valid_loss': '8.71e-03'}
{'epoch': 1, 'train_loss': '4.66e-03', 'valid_loss': '3.80e-03'}
{'epoch': 2, 'train_loss': '1.64e-03', 'valid_loss': '1.78e-03'}
{'epoch': 3, 'train_loss': '9.38e-04', 'valid_loss': '1.41e-03'}
{'epoch': 4, 'train_loss': '8.21e-04', 'valid_loss': '1.29e-03'}
{'epoch': 5, 'train_loss': '7.56e-04', 'valid_loss': '1.19e-03'}
{'epoch': 6, 'train_loss': '7.04e-04', 'valid_loss': '1.11e-03'}
{'epoch': 7, 'train_loss': '6.61e-04', 'valid_loss': '1.04e-03'}
{'epoch': 8, 'train_loss': '6.24e-04', 'valid_loss': '9.69e-04'}
{'epoch': 9, 'train_loss': '5.94e-04', 'valid_loss': '9.08e-04'}
{'epoch': 10, 'train_loss': '5.69e-04', 'valid_loss': '8.53e-04'}
{'epoch': 11, 'train_loss': '5.47e-04', 'valid_loss': '8.03e-04'}
{'epoch': 12, 'train_loss': '5.29e-04', 'valid_loss': '7.57e-04'}
{'epoch': 13, 'train_loss': '5.14e-04', 'valid_loss': '7.16e-04'}
{'epoch': 14, 'train_loss': '5.01e-04', 'valid_loss': '6.79e-04'}
{'epoch': 15, 'train

For Adam optimizer, we also tried the learning rate = 0.1, 0.01, 0.001, and 0.0001. Speaking of the result, lr = 0.0001 gives the best result

For Adam optimizer, we also tried the weight decay = 0.8, 0.85 and 0.9. Speaking of the result, no decay is the best match

The best loss gained from Adam ( without changing batch size.etc) is 3.03e-04

Hyperparameter: changing hidden size.

In [ ]:
model = Model(hidden_size=64)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_function = nn.MSELoss()

min_loss = float('inf')
count = 0

for epoch in range(100):
    train_loss = 0.0

    for i in range(0, len(train_X), 16):
        output = model(torch.from_numpy(train_X[i:i+16]).float())
        loss = loss_function(output, torch.from_numpy(train_y[i:i+16]).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= (len(train_X)//16)

    # validation
    with torch.no_grad():
        valid_loss = 0.0
        for i in range(0, len(valid_X), 16):
            output = model(torch.from_numpy(valid_X[i:i+16]).float())
            loss = loss_function(output, torch.from_numpy(valid_y[i:i+16]).float())
            valid_loss += loss.item()
    valid_loss /= (len(valid_X)//16)

    print({ 'epoch': epoch, 'train_loss': '{:.2e}'.format(train_loss), 
    'valid_loss': '{:.2e}'.format(valid_loss),})

    # early stopping
    if valid_loss > min_loss:
        count += 1
        if count > 9:
            break
    else:
        min_loss = valid_loss
        count = 0
        
print(f'best score = {min_loss:.2e}')

{'epoch': 0, 'train_loss': '3.61e-03', 'valid_loss': '1.00e-03'}
{'epoch': 1, 'train_loss': '6.17e-04', 'valid_loss': '8.17e-04'}
{'epoch': 2, 'train_loss': '5.59e-04', 'valid_loss': '7.22e-04'}
{'epoch': 3, 'train_loss': '5.17e-04', 'valid_loss': '6.52e-04'}
{'epoch': 4, 'train_loss': '4.88e-04', 'valid_loss': '5.89e-04'}
{'epoch': 5, 'train_loss': '4.66e-04', 'valid_loss': '5.37e-04'}
{'epoch': 6, 'train_loss': '4.50e-04', 'valid_loss': '4.98e-04'}
{'epoch': 7, 'train_loss': '4.38e-04', 'valid_loss': '4.67e-04'}
{'epoch': 8, 'train_loss': '4.28e-04', 'valid_loss': '4.43e-04'}
{'epoch': 9, 'train_loss': '4.21e-04', 'valid_loss': '4.24e-04'}
{'epoch': 10, 'train_loss': '4.14e-04', 'valid_loss': '4.09e-04'}
{'epoch': 11, 'train_loss': '4.09e-04', 'valid_loss': '3.96e-04'}
{'epoch': 12, 'train_loss': '4.04e-04', 'valid_loss': '3.85e-04'}
{'epoch': 13, 'train_loss': '4.00e-04', 'valid_loss': '3.76e-04'}
{'epoch': 14, 'train_loss': '3.96e-04', 'valid_loss': '3.68e-04'}
{'epoch': 15, 'train

Hyperparameter: number of layers

In [ ]:
model = Model(num_layers=3)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_function = nn.MSELoss()

min_loss = float('inf')
count = 0

for epoch in range(100):
    train_loss = 0.0

    for i in range(0, len(train_X), 16):
        output = model(torch.from_numpy(train_X[i:i+16]).float())
        loss = loss_function(output, torch.from_numpy(train_y[i:i+16]).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= (len(train_X)//16)

    # validation
    with torch.no_grad():
        valid_loss = 0.0
        for i in range(0, len(valid_X), 16):
            output = model(torch.from_numpy(valid_X[i:i+16]).float())
            loss = loss_function(output, torch.from_numpy(valid_y[i:i+16]).float())
            valid_loss += loss.item()
    valid_loss /= (len(valid_X)//16)

    print({ 'epoch': epoch, 'train_loss': '{:.2e}'.format(train_loss), 
    'valid_loss': '{:.2e}'.format(valid_loss),})

    # early stopping
    if valid_loss > min_loss:
        count += 1
        if count > 9:
            break
    else:
        min_loss = valid_loss
        count = 0
        
print(f'best score = {min_loss:.2e}')

{'epoch': 0, 'train_loss': '1.16e-02', 'valid_loss': '6.57e-03'}
{'epoch': 1, 'train_loss': '3.55e-03', 'valid_loss': '2.12e-03'}
{'epoch': 2, 'train_loss': '1.33e-03', 'valid_loss': '1.10e-03'}
{'epoch': 3, 'train_loss': '9.75e-04', 'valid_loss': '9.33e-04'}
{'epoch': 4, 'train_loss': '8.86e-04', 'valid_loss': '8.64e-04'}
{'epoch': 5, 'train_loss': '8.25e-04', 'valid_loss': '8.09e-04'}
{'epoch': 6, 'train_loss': '7.72e-04', 'valid_loss': '7.61e-04'}
{'epoch': 7, 'train_loss': '7.26e-04', 'valid_loss': '7.18e-04'}
{'epoch': 8, 'train_loss': '6.85e-04', 'valid_loss': '6.80e-04'}
{'epoch': 9, 'train_loss': '6.48e-04', 'valid_loss': '6.47e-04'}
{'epoch': 10, 'train_loss': '6.17e-04', 'valid_loss': '6.18e-04'}
{'epoch': 11, 'train_loss': '5.88e-04', 'valid_loss': '5.92e-04'}
{'epoch': 12, 'train_loss': '5.64e-04', 'valid_loss': '5.69e-04'}
{'epoch': 13, 'train_loss': '5.42e-04', 'valid_loss': '5.49e-04'}
{'epoch': 14, 'train_loss': '5.23e-04', 'valid_loss': '5.31e-04'}
{'epoch': 15, 'train

Hyperparameter: changing dropout rate.

In [ ]:
model = Model(dropout=0.3)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_function = nn.MSELoss()

min_loss = float('inf')
count = 0

for epoch in range(100):
    train_loss = 0.0

    for i in range(0, len(train_X), 16):
        output = model(torch.from_numpy(train_X[i:i+16]).float())
        loss = loss_function(output, torch.from_numpy(train_y[i:i+16]).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= (len(train_X)//16)

    # validation
    with torch.no_grad():
        valid_loss = 0.0
        for i in range(0, len(valid_X), 16):
            output = model(torch.from_numpy(valid_X[i:i+16]).float())
            loss = loss_function(output, torch.from_numpy(valid_y[i:i+16]).float())
            valid_loss += loss.item()
    valid_loss /= (len(valid_X)//16)

    print({ 'epoch': epoch, 'train_loss': '{:.2e}'.format(train_loss), 
    'valid_loss': '{:.2e}'.format(valid_loss),})

    # early stopping
    if valid_loss > min_loss:
        count += 1
        if count > 9:
            break
    else:
        min_loss = valid_loss
        count = 0
        
print(f'best score = {min_loss:.2e}')

{'epoch': 0, 'train_loss': '8.43e-03', 'valid_loss': '4.26e-03'}
{'epoch': 1, 'train_loss': '3.13e-03', 'valid_loss': '1.51e-03'}
{'epoch': 2, 'train_loss': '1.72e-03', 'valid_loss': '9.59e-04'}
{'epoch': 3, 'train_loss': '1.41e-03', 'valid_loss': '8.47e-04'}
{'epoch': 4, 'train_loss': '1.26e-03', 'valid_loss': '7.90e-04'}
{'epoch': 5, 'train_loss': '1.15e-03', 'valid_loss': '7.47e-04'}
{'epoch': 6, 'train_loss': '1.06e-03', 'valid_loss': '7.11e-04'}
{'epoch': 7, 'train_loss': '9.76e-04', 'valid_loss': '6.80e-04'}
{'epoch': 8, 'train_loss': '9.08e-04', 'valid_loss': '6.54e-04'}
{'epoch': 9, 'train_loss': '8.49e-04', 'valid_loss': '6.30e-04'}
{'epoch': 10, 'train_loss': '7.98e-04', 'valid_loss': '6.08e-04'}
{'epoch': 11, 'train_loss': '7.53e-04', 'valid_loss': '5.89e-04'}
{'epoch': 12, 'train_loss': '7.14e-04', 'valid_loss': '5.71e-04'}
{'epoch': 13, 'train_loss': '6.80e-04', 'valid_loss': '5.54e-04'}
{'epoch': 14, 'train_loss': '6.49e-04', 'valid_loss': '5.39e-04'}
{'epoch': 15, 'train

Combining optimimization (best model we currently have).

In [ ]:
model = Model(hidden_size=64, num_layers=3, dropout=0.3)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_function = nn.MSELoss()

min_loss = float('inf')
count = 0

for epoch in range(100):
    train_loss = 0.0

    for i in range(0, len(train_X), 16):
        output = model(torch.from_numpy(train_X[i:i+16]).float())
        loss = loss_function(output, torch.from_numpy(train_y[i:i+16]).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= (len(train_X)//16)

    # validation
    with torch.no_grad():
        valid_loss = 0.0
        for i in range(0, len(valid_X), 16):
            output = model(torch.from_numpy(valid_X[i:i+16]).float())
            loss = loss_function(output, torch.from_numpy(valid_y[i:i+16]).float())
            valid_loss += loss.item()
    valid_loss /= (len(valid_X)//16)

    print({ 'epoch': epoch, 'train_loss': '{:.2e}'.format(train_loss), 
    'valid_loss': '{:.2e}'.format(valid_loss),})

    # early stopping
    if valid_loss > min_loss:
        count += 1
        if count > 9:
            break
    else:
        min_loss = valid_loss
        count = 0
        torch.save(model, 'best_model')
        
print(f'best score = {min_loss:.2e}')

{'epoch': 0, 'train_loss': '6.80e-03', 'valid_loss': '2.46e-03'}
{'epoch': 1, 'train_loss': '1.49e-03', 'valid_loss': '1.00e-03'}
{'epoch': 2, 'train_loss': '1.06e-03', 'valid_loss': '9.33e-04'}
{'epoch': 3, 'train_loss': '9.48e-04', 'valid_loss': '8.52e-04'}
{'epoch': 4, 'train_loss': '8.50e-04', 'valid_loss': '7.87e-04'}
{'epoch': 5, 'train_loss': '7.68e-04', 'valid_loss': '7.29e-04'}
{'epoch': 6, 'train_loss': '7.00e-04', 'valid_loss': '6.79e-04'}
{'epoch': 7, 'train_loss': '6.42e-04', 'valid_loss': '6.34e-04'}
{'epoch': 8, 'train_loss': '5.93e-04', 'valid_loss': '5.93e-04'}
{'epoch': 9, 'train_loss': '5.51e-04', 'valid_loss': '5.55e-04'}
{'epoch': 10, 'train_loss': '5.17e-04', 'valid_loss': '5.21e-04'}
{'epoch': 11, 'train_loss': '4.88e-04', 'valid_loss': '4.89e-04'}
{'epoch': 12, 'train_loss': '4.64e-04', 'valid_loss': '4.61e-04'}
{'epoch': 13, 'train_loss': '4.46e-04', 'valid_loss': '4.37e-04'}
{'epoch': 14, 'train_loss': '4.31e-04', 'valid_loss': '4.16e-04'}
{'epoch': 15, 'train

Summary.

In [ ]:
model = torch.load('best_model')
with torch.no_grad():
    test_loss = 0.0
    for i in range(0, len(test_X), 16):
        output = model(torch.from_numpy(test_X[i:i+16]).float())
        loss = loss_function(output, torch.from_numpy(test_y[i:i+16]).float())
        test_loss += loss.item()
test_loss /= (len(test_X)//16)
print(f'test loss = {test_loss:.2e}')

test loss = 1.86e-03
